In [1]:
from QuantumCircuit import QuantumCircuit as nqc
from utils.GetNoiseModel import GetNoiseModel
import pickle
import json
import numpy as np

In [2]:
my_token = "agSkijZNjBDDI2fALkRUg76VoiGtKr3gTJs9TOiXW5Kr"
backend_name = "ibm_sherbrooke"

noise_model = GetNoiseModel(backend_name=backend_name, token=my_token).get_noise_model()

qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-07-24 18:28:55,145: Default instance not set. Searching all available instances.


In [3]:
circ = nqc(num_qubits=3, noise_model=noise_model, num_trajectories=100, num_cores=5, threshold=1e-12, jsonize=True)

Completed Extraction of ECR Errors.
Starting post-processing on Single Qubit Errors.
Completed post-processing on Single Qubit Errors.
Processing ECR errors.
  Qubit pair (0, 1): 48/48 errors above threshold (0 filtered out)
  Qubit pair (1, 2): 48/48 errors above threshold (0 filtered out)
ECR errors processed.
Building Noise Operators for ECR Errors.
Completed building Noise Operators for ECR Errors.
Extracting Measurement Errors.
Available qubits in roerror_map: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126]
Requ

In [4]:
gate_set_single = {"x": circ.X, "y":circ.Y, "z":circ.Z, "h":circ.H, "sx":circ.SX, "rx":circ.RX, "ry":circ.RY, "rz":circ.RZ}
gate_set_double = [circ.CX, circ.CY, circ.CZ]

def build_random_circuit(depth):
    for _ in range(depth):
        for i in range(3):
            choice = np.random.choice(list(gate_set_single.keys()))
        if choice in ["rx", "ry", "rz"]:
            angle = np.random.uniform(0, 2*np.pi)
            gate_set_single[choice](angle, i)
        else:
            gate_set_single[choice](i)
        for i in range(2):
            np.random.choice(gate_set_double)(i, i+1)
        np.random.choice(gate_set_double)(2, 0)

In [5]:
depth = 20
build_random_circuit(depth)

In [6]:
p_mcwf = circ.execute(qubits=[0,1,2], num_trajectories=100)

In [6]:
p_pure = circ.run_pure_state([0,1,2])

In [7]:
p_ref = circ.run_with_density_matrix([0,1,2])

In [9]:
p_mcwf1k = circ.execute(qubits=[0,1,2], num_trajectories=1000)

In [9]:
p_mcwf_1 = circ.execute(qubits=[0,1,2], num_trajectories=1)
p_mcwf_10 = circ.execute(qubits=[0,1,2], num_trajectories=10)

In [10]:
def metrics(p1,p2):
    coeff = np.sum(np.sqrt(p1 * p2))
    dist = -np.log(coeff)
    hellinger = np.sqrt(1 - coeff)
    m = 0.5 * (p1 + p2)
    js = 0.5 * np.sum(p1 * np.log(1e-10 + p1 / m) + p2 * np.log(1e-10 + p2 / m))
    print(f"Coefficient: {coeff}, Distance: {dist}, Hellinger: {hellinger}, Jensen-Shannon: {js}")

In [11]:
print("Metrics for MCWF (1 Trajectory):")
metrics(p_mcwf_1, p_ref)
print("====================================================")
print("Metrics for MCWF (10 Trajectories):")
metrics(p_mcwf_10, p_ref)
# print("====================================================")
# print("Metrics for MCWF (100 Trajectories):")
# metrics(p_mcwf, p_ref)
# print("====================================================")
# print("Metrics for MCWF (1000 Trajectories):")
# metrics(p_mcwf1k, p_ref)
print("====================================================")
print("Metrics for Pure State:")
metrics(p_pure, p_ref)

Metrics for MCWF (1 Trajectory):
Coefficient: 0.5622925847588569, Distance: 0.5757329511106878, Hellinger: 0.6615946003718162, Jensen-Shannon: 0.36111480030868015
Metrics for MCWF (10 Trajectories):
Coefficient: 0.9683953935672736, Distance: 0.032114810675443764, Hellinger: 0.1777768444784822, Jensen-Shannon: 0.0313306147280001
Metrics for Pure State:
Coefficient: 0.8065824955628269, Distance: 0.2149490982873674, Hellinger: 0.4397925697839529, Jensen-Shannon: 0.14884328342301517
